* This notebook is to demonstrate how to use Bayesian Optimization package on github <https://github.com/fmfn/BayesianOptimization> locally to tune hyperparamters for our models(LR, embedding dimension=1) for predicting heart failure onset risk on cerner sample data
* For this demonstration, the data is the original 1 hospital (h143) previously used by retain, with 42,729 patients in total
* The logistic regression model has the architecture of an embedding layer (embedding dimension =1), a linear activation and sigmoid transformation. The hyperparameters to be tuned are: learning rate: lr, and l2 regularization 
* To implement this, first you need to install the package: however we modify the package file a bit to bypass error and keep on iterating. The modified files could be found at Experiments/modifiedBO
* Then **important**: you need to define a function (in our case LR_tune()) which takes in the hyperparameters: l2, lr on logscale, run the model using models, Loaddata, and TrainVaTe modules and return the best validation auc
* Be ware that this BO package will search float parameters, so if you have int or categorical parameters you want to tune, you might want to transform those values in your function before giving those to your models
* Then **important**: call BO function and pass your LR_tune(), a search range for each parameter ((-16, 1) means -16 and 1 inclusive), and give it points to explore (points that will give you large target values) if you want to, and call maximize() and pass number of iterations you want to run BO
* Then you will get results of your initial designated explored points(if any), 5 initializations, and plus number of BO iterations
* For our results: it improved our best validation auc **from manually tuned 0.76119** (l2 = np.exp(-11), lr = np.exp(-9)) to **0.78456** (l2 = np.exp(-8.4004), lr = np.exp(--6.1833))

In [1]:
from __future__ import print_function
from __future__ import division

import string
import re
import random

import os
import sys
import argparse
import time
import math

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torchviz import make_dot, make_dot_from_trace

from sklearn.metrics import roc_auc_score  
from sklearn.metrics import roc_curve 

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

try:
    import cPickle as pickle
except:
    import pickle


from bayes_opt import BayesianOptimization

In [2]:
import model_lr1 as model #here the em experiment
import Loaddata as Loaddata
import TrainVaTe_lr1 as TVT

# check GPU availability
use_cuda = torch.cuda.is_available()
use_cuda

True

In [3]:
# Load data set and target values
"""
data, target = make_classification(
    n_samples=1000,
    n_features=45,
    n_informative=12,
    n_redundant=7
)
"""
set_x = pickle.load(open('Data/h143.visits', 'rb'), encoding='bytes')
set_y = pickle.load(open('Data/h143.labels', 'rb'),encoding='bytes')

model_x = []
for patient in set_x:
    model_x.append([each for visit in patient for each in visit])  
    
merged_set= [[set_y[i],model_x[i]] for i in range(len(set_y))] #list of list or list of lists of lists
print("\nLoading and preparing data...")    
train1, valid1, test1 = Loaddata.load_data(merged_set)
print("\nSample data after split:")  
print(train1[0])
print("model is", 'LR') #can change afterwards


Loading and preparing data...

Sample data after split:
[0, [1667, 144, 62, 85, 1667, 144, 62, 85]]
model is LR


In [4]:
#function to record comprehensive searching results 
def print2file(buf, outFile):
    outfd = open(outFile, 'a')
    outfd.write(buf + '\n')
    outfd.close()

logFile='testLR_dim1.log'
header = 'model|l2|lr|BestValidAUC|TestAUC|atEpoch'
print2file(header, logFile)

In [10]:
#Hyperparamters to tune for LR: embed_dim, l2, lr. Define a function to return the validation AUC of the model 
def LR_tune( l2, lr):
    #little transformations to use the searched values
    #embed_dim = 2 ** int(embed_dim) #base 2 
    l2 = np.exp(l2) #base e
    lr = np.exp(lr) #base e
    ehr_model = model.EHR_LR(input_size =20000)  
    if use_cuda:
        ehr_model = ehr_model.cuda()
    optimizer = optim.Adam(ehr_model.parameters(), lr=lr, weight_decay=l2)
    #current_loss_allep=[]
    #all_losses_allep=[]
    #avg_losses_allep=[]
    #train_auc_allep =[]
    #valid_auc_allep =[]
    #test_auc_allep=[]
    
    bestValidAuc = 0.0
    bestTestAuc = 0.0
    bestValidEpoch = 0
    
    for ep in range(25):
        #start = time.time()
        current_loss, train_loss, _ = TVT.train(train1, model= ehr_model, optimizer = optimizer, batch_size = 1) #mb=args.mb
        avg_loss = np.mean(train_loss)
        #train_time = timeSince(start)
        #print ('\n Current running on: Epoch ', ep,'Training loss:',' Average loss', avg_loss)
        #print(train_loss, '\n train loss plot')
        #TVT.showPlot(train_loss)
        #eval_start = time.time()
        #train_auc, y_real, y_hat = TVT.calculate_auc(model= ehr_model, data = train1, which_model ='LR', batch_size = 1)
        #print ('\n Current running on: Epoch ', ep,' Training auc:', train_auc)
        #TVT.auc_plot(y_real, y_hat)
        valid_auc, y_real, y_hat, _  = TVT.calculate_auc(model = ehr_model, data = valid1, which_model = 'LR', batch_size = 1)
        #print ('\n Current running on: Epoch ', ep,' validation auc:', valid_auc)
        #TVT.auc_plot(y_real, y_hat)
        if valid_auc > bestValidAuc: 
            bestValidAuc = valid_auc
            bestValidEpoch = ep
            bestTestAuc, y_real, y_hat,_ = TVT.calculate_auc(model = ehr_model, data = test1, which_model = 'LR', batch_size = 1)

        if ep - bestValidEpoch >10:
            break
        
      
        buf = '|%f |%f |%d ' % (bestValidAuc, bestTestAuc, bestValidEpoch)
        pFile= 'LR'+'|'+str(l2)+'|'+str(lr)+'|'+buf    
        print2file(pFile, logFile)      
        
        #test_auc, y_real, y_hat = TVT.calculate_auc(model = ehr_model, data = test1, which_model = 'LR', batch_size = 1)
        #print ('\n Current running on: Epoch ', ep,' test auc:', test_auc)
        #TVT.auc_plot(y_real, y_hat)
        #eval_time = timeSince(eval_start)
        #print ("Epoch ", ep, "Summary:  Training_auc :", train_auc, " , Validation_auc : ", valid_auc, " ,& Test_auc : " , test_auc, " Avg Loss: ", avg_loss )
        #current_loss_allep.append(current_loss)
        #all_losses_allep.append(train_loss)
        #avg_losses_allep.append(avg_loss)
        #train_auc_allep.append(train_auc)
        #valid_auc_allep.append(valid_auc)
        #test_auc_allep.append(test_auc)
        #final_max_valid = max(valid_auc_allep)
    return bestValidAuc

In [12]:
if __name__ == "__main__":
    gp_params = {"alpha": 1e-4}

    LRBO = BayesianOptimization(LR_tune,
        {'l2': (-16, 1), 'lr': (-11, -2) })
    LRBO.explore({'l2': [-11], 'lr': [-9]})

    LRBO.maximize(n_iter=30, **gp_params)

    print('-' * 53)
    print('Final Results')
    print('LR: %f' % LRBO.res['max']['max_val'])

Initialization
-----------------------------------------------------
 Step |   Time |      Value |        l2 |        lr | 
Parameter containing:
-6.4644e-01
-7.6218e-02
 7.4080e-01
     ⋮      
-4.9225e-01
 6.0644e-01
 2.4525e-01
[torch.FloatTensor of size 20000x1]

        emb                           types
0 -0.646437                            Code
1 -0.076218  b'M_ACETAMINOPHEN-HYDROCODONE'
2  0.740798         b'M_BACITRACIN TOPICAL'
3 -0.325168    b'M_BUPIVACAINE-EPINEPHRINE'
4  0.170204                  b'M_CEFAZOLIN'
    1 | 21m52s |    0.76119 |  -11.0000 |   -9.0000 | 
Parameter containing:
-0.5830
 0.0702
-0.7913
   ⋮    
-1.2227
-0.8947
-0.2982
[torch.FloatTensor of size 20000x1]

        emb                           types
0 -0.582957                            Code
1  0.070173  b'M_ACETAMINOPHEN-HYDROCODONE'
2 -0.791270         b'M_BACITRACIN TOPICAL'
3 -1.034595    b'M_BUPIVACAINE-EPINEPHRINE'
4 -1.137885                  b'M_CEFAZOLIN'
    2 | 20m12s |    0.75781 |   -

Parameter containing:
-1.5167e-01
 4.4071e-01
-1.9550e+00
     ⋮      
-7.1001e-01
-9.8930e-01
-1.7379e+00
[torch.FloatTensor of size 20000x1]

        emb                           types
0 -0.151665                            Code
1  0.440711  b'M_ACETAMINOPHEN-HYDROCODONE'
2 -1.955047         b'M_BACITRACIN TOPICAL'
3 -0.600602    b'M_BUPIVACAINE-EPINEPHRINE'
4 -0.537198                  b'M_CEFAZOLIN'
   18 | 25m04s |    0.78002 |   -5.7732 |   -7.7989 | 
Parameter containing:
-2.4210e-01
 5.3678e-01
-1.4415e-01
     ⋮      
-7.8548e-01
-4.8847e-01
-1.4422e+00
[torch.FloatTensor of size 20000x1]

        emb                           types
0 -0.242096                            Code
1  0.536776  b'M_ACETAMINOPHEN-HYDROCODONE'
2 -0.144147         b'M_BACITRACIN TOPICAL'
3  1.113546    b'M_BUPIVACAINE-EPINEPHRINE'
4 -1.519608                  b'M_CEFAZOLIN'
   19 | 18m17s |    0.78304 |   -9.5729 |   -5.9139 | 
Parameter containing:
-4.4124e-01
 1.9576e+00
 6.9730e-01
     ⋮      
 1.

/home/jzhu8/.local/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-2.65402039e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)


Parameter containing:
-7.6477e-01
 2.5619e-02
 1.4927e+00
     ⋮      
-7.7463e-01
 7.6500e-01
-1.6530e+00
[torch.FloatTensor of size 20000x1]

        emb                           types
0 -0.764771                            Code
1  0.025619  b'M_ACETAMINOPHEN-HYDROCODONE'
2  1.492747         b'M_BACITRACIN TOPICAL'
3  1.932553    b'M_BUPIVACAINE-EPINEPHRINE'
4  0.223412                  b'M_CEFAZOLIN'
   21 | 12m22s |    0.53702 |   -5.0580 |   -2.0000 | 
Parameter containing:
-1.2783e+00
-1.2361e+00
 2.4593e-01
     ⋮      
-1.1686e+00
-2.6337e-01
-1.1037e+00
[torch.FloatTensor of size 20000x1]

        emb                           types
0 -1.278262                            Code
1 -1.236094  b'M_ACETAMINOPHEN-HYDROCODONE'
2  0.245931         b'M_BACITRACIN TOPICAL'
3  0.529936    b'M_BUPIVACAINE-EPINEPHRINE'
4  1.375595                  b'M_CEFAZOLIN'
   22 | 11m27s |    0.75288 |  -16.0000 |   -4.9168 | 
Parameter containing:
-4.3772e-01
-2.8627e+00
-5.2456e-01
     ⋮      
-8.

/home/jzhu8/.local/lib/python3.5/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-2.54075366e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 46, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


Parameter containing:
 1.5299e+00
-1.7964e-02
 1.0447e-01
     ⋮      
-7.1884e-01
-1.6650e+00
-8.4150e-01
[torch.FloatTensor of size 20000x1]

        emb                           types
0  1.529939                            Code
1 -0.017964  b'M_ACETAMINOPHEN-HYDROCODONE'
2  0.104468         b'M_BACITRACIN TOPICAL'
3  0.172383    b'M_BUPIVACAINE-EPINEPHRINE'
4  1.079036                  b'M_CEFAZOLIN'
   27 | 13m59s |    0.77841 |   -7.1544 |   -7.8085 | 
Parameter containing:
-1.9835e-02
 1.0592e+00
-2.2739e+00
     ⋮      
-8.4625e-01
 1.1436e+00
 1.7440e-03
[torch.FloatTensor of size 20000x1]

        emb                           types
0 -0.019835                            Code
1  1.059249  b'M_ACETAMINOPHEN-HYDROCODONE'
2 -2.273923         b'M_BACITRACIN TOPICAL'
3 -0.119679    b'M_BUPIVACAINE-EPINEPHRINE'
4  0.752667                  b'M_CEFAZOLIN'
   28 | 14m57s |    0.77133 |   -4.8923 |   -7.7471 | 
Parameter containing:
 1.5201e+00
 9.1182e-01
-4.5721e-01
     ⋮      
-9.

In [13]:
np.exp(-8.4004)

0.00022477739523616444

In [14]:
np.exp(-6.1833)

0.002063606703809012